# Greedy Algorithm for Motif Finding

This is an approximate greedy algorithm. Although the approximation ratio is unknown, it often performs well in practice, and is quite simple.

In [1]:
include("utils.jl")
# utils.jl has the align, getprofile, and score utility functions:

seqs = ["AGTGGATACC", # motif GGA inserted at 4
        "AGGATATAGT", # 2
        "AGGAGGATAT", # 5
        "GCAGCGGATA", # 5
        "CATCAGGATA"] # 6
s = [1, 1, 1, 1, 1]
l = 5;

In [2]:
function motifsearch_greedy(sequences, l)
    """
    Given a 2D array of strings *sequences*, greedy 
    search for a motif of length *l*.
    Return an array of indices corresponding to the 
    alignment found with the highest score.
    """
    # TODO: Check that l <= sequence length
    bestmotif = ones(Int32, 2)
    
    for s₁ = 1:length(sequences[1])-l+1
        for s₂ = 1:length(sequences[2])-l+1
            # find best partial alignment of first 2 sequences 
            if score(sequences, [s₁, s₂], l) > score(sequences, bestmotif, l)
                bestmotif = [s₁, s₂]
            end
        end
    end
    s = copy(bestmotif)
    
    #iterate through the rest of the sequences
    for i = 3:length(sequences)
        # extend the default best motif
        push!(bestmotif, 1)
        for sᵢ = 1:length(sequences[i])-l+1
            if score(sequences, [s..., sᵢ], l) > score(sequences, bestmotif, l)
                # update the best motif
                bestmotif[i] = sᵢ
            end
        end
        push!(s, bestmotif[i])
    end
    return bestmotif
end

motifsearch_greedy (generic function with 1 method)

In [3]:
motif = motifsearch_greedy(seqs, l)
alignment = align(seqs, motif, l)
profile = getprofile(alignment)
display(motif)
display(alignment)
display(profile)
score(profile)

5-element Array{Int64,1}:
 4
 2
 5
 6
 6

5-element Array{Any,1}:
 "GGATA"
 "GGATA"
 "GGATA"
 "GGATA"
 "GGATA"

4×5 Array{Int32,2}:
 0  0  5  0  5
 0  0  0  5  0
 0  0  0  0  0
 5  5  0  0  0

25

## Evaluating the Algorithm
Simple performance metric for this greedy algorithm on synthetic data.

The index error is the average number of indices that do not match the actual indices of the inserted motifs.

The score error is the average score given to profiles found by motifsearch_greedy minus the maximum possible score for a correct alignment.

In [6]:
include("data.jl")

errs = []
score_errs = []
t = 20
n = 300
l = 15
max_score = l*t
for i = 1:20
    # 10 sequences of length 20, motiflen 6, no mutations
    seqs, motif, indices = gendata(t, n, l, 0)
    predicted_indices = motifsearch_greedy(seqs, l)
    err = sum(indices .!= predicted_indices)
    score_err = score(seqs, predicted_indices, l) - max_score
    push!(errs, err)
    push!(score_errs, score_err)
end
@printf("Mean absolute index error: %f\n", mean(errs))
@printf("Mean absolute score error: %f\n", mean(score_errs))

Mean absolute index error: 6.000000
Mean absolute score error: -4.900000
